In [1]:
import andes
from andes.interop.pandapower import to_pandapower, make_GSF
import pandapower as pp

from numpy import pi
from andes.shared import pd, rad2deg, deg2rad

import numpy as np
import andes.interop.pandapower as adpp

import pandas as pd

import andes
import numpy as np
from andes.interop.pandapower import to_pandapower, make_link_table, runopp_map, build_group_table
from andes.interop.pandapower import add_gencost, build_group_table
andes.config_logger(stream_level=20)

import pandas as pd
import pandapower as pp
from math import ceil, floor

import os

import matplotlib.pyplot as plt

import scipy.stats as stat

from jams import rted2, dcopf, rted3

In [2]:
# !andes run '/Users/jinningwang/Documents/work/andes/jwang/case/case2000.m' --convert

In [3]:
print(andes.__version__)
print(pp.__version__)

1.7.1.post223.dev0+g89ef79bf
2.8.0


In [4]:
# ss0 is used for PP conversion

case2k = '/Users/jinningwang/Documents/work/andes/jwang/case/case2000.xlsx'
case14 = andes.get_case('ieee14/ieee14_ac8b.xlsx')
wecc = andes.get_case('wecc/wecc_full.xlsx')
npcc = andes.get_case('npcc/npcc.xlsx')
case14 = '/Users/jinningwang/Documents/work/andes/jwang/case/ieee14_pp.xlsx'
ssa = andes.load(case14,
                 setup=True,
                 no_output=True,
                 default_config=False)

Working directory: "/Users/jinningwang/Documents/work/andes/jwang/notes2"
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Generated code for <PQ> is stale.
Numerical code generation (rapid incremental mode) started...


Generating code for 1 models on 8 processes.


Saved generated pycode to "/Users/jinningwang/.andes/pycode"
> Reloaded generated Python code of module "pycode".
Generated numerical code for 1 models in 0.1899 seconds.
Parsing input file "/Users/jinningwang/Documents/work/andes/jwang/case/ieee14_pp.xlsx"...
Input file parsed in 0.0621 seconds.
System internal structure set up in 0.0360 seconds.


In [5]:
# --- conversion ---
ssp = to_pandapower(ssa)

# add gen cost, G1-11. G10 EV, G11 Slack
linearcost = [1] * ssp.gen.shape[0]

gen_cost = np.array([[2., 0., 0., 3., 0., 0., 0.]] * ssp.gen.shape[0])
gen_cost[:, 5] = linearcost  # c1

add_gencost(ssp, gen_cost)

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0038 seconds.
0: |F(x)| = 0.5605182607
1: |F(x)| = 0.006202201971
2: |F(x)| = 5.819385332e-06
3: |F(x)| = 6.964193111e-12
Converged in 4 iterations in 0.0097 seconds.
Power flow results are consistent. Conversion is successful.


True

In [6]:
ssa_key = make_link_table(ssa)

In [7]:
ssp.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,40.000000,30.436143,-1.764070,1.03
1,40.000000,12.597131,-3.537129,1.01
2,30.000000,20.986594,-6.452736,1.03
3,35.000000,7.396392,-1.539957,1.03
4,81.427213,-21.617097,0.000000,1.03


In [9]:
runopp_map(ssp, ssa_key)

,stg_idx,p,q,vm_pu,bus_idx,controllable,dg_idx,syg_idx,gov_idx,exc_idx
0,2,0.250537,0.114989,1.095812,2,True,NaN,GENROU_2,TGOV1_2,EXST1_1
1,3,0.499999,0.149999,1.085312,3,True,NaN,GENROU_3,TGOV1_3,ESST3A_3
2,4,0.499999,0.099999,1.090565,6,True,False,False,NaN,NaN
3,5,0.5,0.080081,1.099998,8,True,NaN,GENROU_5,TGOV1_5,1
4,1,0.5,-0.075928,1.100000,1,True,NaN,GENROU_1,TGOV1_1,ESST3A_2


In [10]:
ssp.res_cost

225.05353523016953

In [11]:
ssd = dcopf()
ssd.from_andes(ssa)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-08


-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0034 seconds.
0: |F(x)| = 0.5605182607
1: |F(x)| = 0.006202201971
2: |F(x)| = 5.819385332e-06
3: |F(x)| = 6.964193111e-12
Converged in 4 iterations in 0.0048 seconds.
Power flow results are consistent. Conversion is successful.
GSF is consistent.
/Users/jinningwang/Documents/work/mambaforge/envs/andes/lib/python3.9/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [12]:
ssd.get_res()

Successfully build DCOPF model.


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 33 rows, 5 columns and 101 nonzeros
Model fingerprint: 0x5316914a
Coefficient statistics:
  Matrix range     [4e-04, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 2e+00]
  RHS range        [2e+00, 2e+01]
Presolve removed 33 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2370000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.237000000e+00


Successfully solve DCOPF.


,gen,pg
0,2,0.100
1,3,0.100
2,4,0.100
3,5,0.100
4,1,1.837
